#5.5 File I/O

So far we discussed how to process data, how to build, train and test deep learning models. However, at
some point we are likely happy with what we obtained and we want to save the results for later use and
distribution. Likewise, when running a long training process it is best practice to save intermediate results
(checkpointing) to ensure that we don’t lose several days worth of computation when tripping over the
power cord of our server. At the same time, we might want to load a pretrained model (e.g. we might
have word embeddings for English and use it for our fancy spam classifier). For all of these cases we need
to load and store both individual weight vectors and entire models. This section addresses both issues.

#5.5.1 Tensors

In its simplest form, we can directly use the save and load functions to store and read tensors
separately. This works just as expected.


In [0]:
import torch
from torch import nn
x=torch.arange(4,dtype = torch.float32)
torch.save(x,"x-file")

Then, we read the data from the stored file back into memory.

In [10]:
x2 = torch.load("x-file")
x2

tensor([0., 1., 2., 3.])

We can also store a list of tensors and read them back into memory.


In [11]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0., 1., 2., 3.]), tensor([0., 0., 0., 0.]))

We can even write and read a dictionary that maps from a string to an tensors. This is convenient, for
instance when we want to read or write all the weights in a model.

In [12]:
mydict = {'x': x, 'y': y}
torch.save( mydict,'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0., 1., 2., 3.]), 'y': tensor([0., 0., 0., 0.])}

#5.5.2 Torch Model Parameters

Saving individual weight vectors (or other  tensors) is useful but it gets very tedious if we want
to save (and later load) an entire model. After all, we might have hundreds of parameter groups sprinkled
throughout. Writing a script that collects all the terms and matches them to an architecture is quite some
work. For this reason torch provides built-in functionality to load and save entire networks rather than
just single weight vectors. An important detail to note is that this saves model parameters and not the
entire model. I.e. if we have a 3 layer MLP we need to specify the architecture separately. The reason
for this is that the models themselves can contain arbitrary code, hence they cannot be serialized quite so
easily (there is a way to do this for compiled models - please refer to the torch documentation for the
technical details on it). The result is that in order to reinstate a model we need to generate the architecture
in code and then load the parameters from disk. The deferred initialization is quite advantageous here
since we can simply define a model without the need to put actual values in place. Let’s start with our
favorite MLP.


In [0]:
class MLP(nn.Module):
    def __init__(self):
                super().__init__()
                self.hidden = nn.Linear(20, 256)
                self.output = nn.Linear(256, 10)
                self.relu = nn.ReLU()
    def forward(self, x):
        H_1 = self.relu(self.hidden(x))
        out = self.output(H_1)
        return out
    

net = MLP()
x = torch.randn(size=(2, 20))
y = net(x)

Next, we store the parameters of the model as a file with the name mlp.params’

In [0]:
torch.save(net.state_dict(), 'mlp.params')



To check whether we are able to recover the model we instantiate a clone of the original MLP model.
Unlike the random initialization of model parameters, here we read the parameters stored in the file directly.

In [16]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
  (relu): ReLU()
)

Since both instances have the same model parameters, the computation result of the same input x should
be the same. Let’s verify this.

In [17]:
yclone = clone(x)
yclone == y

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=torch.uint8)

#Summary
• The save and load functions can be used to perform File I/O for tensors objects.
• The load_parameters and save_parameters functions allow us to save entire sets of
parameters for a network in torch.
• Saving the architecture has to be done in code rather than in parameters.
#Exercises
1. Even if there is no need to deploy trained models to a different device, what are the practical benefits
of storing model parameters?
2. Assume that we want to reuse only parts of a network to be incorporated into a network of a different
architecture. How would you go about using, say the first two layers from a previous network in a
new network.
3. How would you go about saving network architecture and parameters? What restrictions would
you impose on the architecture?
